In [1]:
import pandas as pd,numpy as np,tensorflow as tf
train=pd.read_csv(r'C:\Users\hp\Desktop\CompetitionData\105COLS\TrainData24DAYS105cols.csv')
test=pd.read_csv(r'C:\Users\hp\Desktop\CompetitionData\105COLS\TestData24DAYS105cols.csv')
col=[x for x in train.columns if x not in ['user_id','label']]
data=train[col]
label=train['label']
data.columns[6],data.columns[47],data.columns[53],data.shape

('register_type', 'no_act_video', 'bias_or_not', (40518, 105))

In [ ]:
from sklearn import preprocessing
ohe1=preprocessing.OneHotEncoder(n_values=[2],categorical_features=[0])
label=label.reshape(-1,1)
label=ohe1.fit_transform(label).toarray()
label.shape

In [ ]:
from sklearn import preprocessing
ohe=preprocessing.OneHotEncoder(n_values=[12,2,2,2,2,2,2,2],categorical_features=[6,47,48,49,50,51,52,53])
data=ohe.fit_transform(data).toarray()

In [ ]:
col1=[x for x in test.columns if x not in ['user_id']]
testdata=test[col1]
testdata.columns[6],testdata.columns[47],testdata.columns[53],testdata.shape
testdata=ohe.fit_transform(testdata).toarray()
testdata.shape,data.shape,label.shape

训练集、测试集预处理完毕

In [ ]:
learning_rate = 1e-3
num_units = 256
num_layer = 3
input_size = 41
time_step = 3
total_steps = 1000
category_num = 2
steps_per_validate = 100
#steps_per_test = 500
batch_size = tf.placeholder(tf.int32, [])
keep_prob = tf.placeholder(tf.float32, [])

In [ ]:
x = tf.placeholder(tf.float32, [None, 123])
y_label = tf.placeholder(tf.float32, [None, 2])
#每个样本123维，分成3个时间序列，每个时间序列41个属性输入网络
x_shape = tf.reshape(x, [-1, time_step, input_size])
#DroupoutWrapper防止过拟合
from tensorflow.contrib.rnn import DropoutWrapper
def cell(num_units):
    cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=num_units)
    return DropoutWrapper(cell, output_keep_prob=keep_prob)

In [ ]:
#三层LSTM
cells = tf.nn.rnn_cell.MultiRNNCell([cell(num_units) for _ in range(num_layer)])
#batch_size=1000，每一个批次的每个样本都赋予一个初始化0状态
h0 = cells.zero_state(batch_size, dtype=tf.float32)
output, hs = tf.nn.dynamic_rnn(cells, inputs=x_shape, initial_state=h0)
#网络中每个样本最后一个时间序列的结果作为输出
output = output[:, -1, :]
# Output Layer，全连接层线性变换
w = tf.Variable(tf.truncated_normal([num_units, category_num], stddev=0.1), dtype=tf.float32)
b = tf.Variable(tf.constant(0.1, shape=[category_num]), dtype=tf.float32)
y = tf.matmul(output, w) + b
# Loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y))

In [ ]:
#生成批次数据
def gen_batch(data,label,batch_size):
    row, col = data.shape
    n = row // batch_size     
    for i in range(n):
        x = data[i * batch_size : (i + 1) * batch_size,:]     # 0~col-1为数据
        y = label[i * batch_size : (i + 1) * batch_size,:]  # 最后一列列是标签
        yield (x, y)

In [ ]:
training= tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

correction_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_label, axis=1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))

saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(total_steps + 1):
        for batch_x,batch_y in gen_batch(data,label,1000):
            sess.run(training, feed_dict={x: batch_x, y_label: batch_y, keep_prob: 0.5, batch_size:1000})
            if step % steps_per_validate == 0:
                print('Train', step, sess.run(accuracy, feed_dict={x: batch_x, y_label: batch_y, keep_prob:0.5,
                                                               batch_size:1000}))
    saver.save(sess,r"C:\Users\hp\tfModels\LSTMdasai.cpkt")